# Week 5, Assignment C1

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf


#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'AAPL'

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, '2000-01-01', '2021-10-31')
df = df.iloc[:, :4]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
1999-12-31,0.901228,0.918527,0.888393,0.917969
2000-01-03,0.936384,1.004464,0.907924,0.999442
2000-01-04,0.966518,0.987723,0.903460,0.915179
2000-01-05,0.926339,0.987165,0.919643,0.928571
2000-01-06,0.947545,0.955357,0.848214,0.848214
...,...,...,...,...
2021-10-25,148.679993,149.369995,147.619995,148.639999
2021-10-26,149.330002,150.839996,149.009995,149.320007
2021-10-27,149.360001,149.729996,148.490005,148.850006


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window = 15).mean()
df['Open_1_Change'] = df['Open'] - df['Open'].shift(1)
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2000-01-21,1.020089,1.020089,0.983817,0.993862,0.908296,-0.011161
2000-01-24,0.968192,1.006696,0.938616,0.948661,0.910342,-0.051897
2000-01-25,0.937500,1.010045,0.914063,1.002232,0.910528,-0.030692
2000-01-26,0.982143,1.019531,0.979911,0.983817,0.915104,0.044643
2000-01-27,0.971540,1.008929,0.955357,0.982143,0.918676,-0.010603
...,...,...,...,...,...,...
2021-10-25,148.679993,149.369995,147.619995,148.639999,144.967331,-1.010010
2021-10-26,149.330002,150.839996,149.009995,149.320007,145.514665,0.650009
2021-10-27,149.360001,149.729996,148.490005,148.850006,145.971332,0.029999


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.


y = np.where(df['Close'].shift(-1) - df['Close'] >= 0, 1, -1)
y

array([-1,  1, -1, ...,  1, -1, -1])

In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.
from sklearn.model_selection import train_test_split
index = int(0.7*len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= int(0.7*len(X)))

In [18]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

In [19]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[-0.020202830555873817]
1,High,[0.09059698483253334]
2,Low,[0.28631018366944766]
3,Close,[-0.37967792211435225]
4,Close_15_Rolling,[0.025637764388045575]
5,Open_1_Change,[-0.1980211527259976]


In [20]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)


In [21]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, predictions)
print(matrix)

[[ 150 1678]
 [ 160 1847]]


In [22]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

print(logistic.score(X_test, y_test))

0.5207301173402868


In [23]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=5)
print(cross_val)
print(cross_val.mean())

[0.52554745 0.52737226 0.5310219  0.53284672 0.52737226]
0.5288321167883212
